In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

# Import the helper function to update the H5 file.
from update_h5 import create_updated_h5

# -------------------------
# Section 1: Shift Centers in CSV
# -------------------------

# File chooser widget for selecting the input CSV file.
csv_file_chooser = FileChooser(os.getcwd())
csv_file_chooser.title = "Select Input CSV File"
csv_file_chooser.filter_pattern = "*.csv"

# Widgets for shift parameters.
shift_x_widget = widgets.FloatText(
    value=0,
    description="Shift X:",
    layout=widgets.Layout(width="200px")
)
shift_y_widget = widgets.FloatText(
    value=0,
    description="Shift Y:",
    layout=widgets.Layout(width="200px")
)

# Button to trigger the shifting process.
process_csv_button = widgets.Button(
    description="Shift Centers and Save CSV",
    button_style="primary"
)

# Output widget for CSV shifting logs.
csv_output = widgets.Output(layout={'border': '1px solid black', 'padding': '5px'})

# Global variable to store the path of the shifted CSV.
shifted_csv_path = None

def on_process_csv_clicked(b):
    global shifted_csv_path
    with csv_output:
        clear_output()
        input_csv = csv_file_chooser.selected
        if not input_csv:
            print("Please select an input CSV file.")
            return

        shift_x = shift_x_widget.value
        shift_y = shift_y_widget.value
        
        # Compute output CSV path.
        shifted_csv_path = os.path.join(
            os.path.dirname(input_csv),
            f"centers_shifted_{shift_x}_{shift_y}.csv"
        )
        
        try:
            df = pd.read_csv(input_csv)
        except Exception as e:
            print("Error reading CSV:", e)
            return
        
        # Check required columns.
        if 'center_x' not in df.columns or 'center_y' not in df.columns:
            print("CSV must contain 'center_x' and 'center_y' columns.")
            return
        
        # Apply the shifts.
        df['center_x'] = df['center_x'] + shift_x
        df['center_y'] = df['center_y'] + shift_y
        
        # Save the updated DataFrame.
        df.to_csv(shifted_csv_path, index=False)
        print(f"Shifted centers saved to: {shifted_csv_path}")

process_csv_button.on_click(on_process_csv_clicked)

csv_ui = widgets.VBox([
    widgets.HTML("<h2>Shift Centers in CSV</h2>"),
    csv_file_chooser,
    widgets.HBox([shift_x_widget, shift_y_widget]),
    process_csv_button,
    csv_output
])

# -------------------------
# Section 2: Update H5 with Shifted Centers
# -------------------------

# File chooser widget for selecting the H5 image file.
image_file_chooser_h5 = FileChooser(os.getcwd())
image_file_chooser_h5.title = "Select H5 Image File for Updating"
image_file_chooser_h5.filter_pattern = "*.h5"

# Button to trigger the H5 update process.
update_h5_button = widgets.Button(
    description="Update H5 with Shifted Centers",
    button_style="primary"
)

# Output widget for H5 update logs.
h5_output = widgets.Output(layout={'border': '1px solid black', 'padding': '5px'})

def on_update_h5_clicked(b):
    with h5_output:
        clear_output()
        if shifted_csv_path is None:
            print("No shifted CSV available. Please run the CSV shifting process first.")
            return
        
        image_file = image_file_chooser_h5.selected
        if not image_file:
            print("Please select an H5 image file.")
            return
        
        # Compute the output H5 file path.
        new_h5_path = os.path.join(
            os.path.dirname(image_file),
            # os.path.splitext(os.path.basename(image_file))[0] + "_centersshifted.h5"
            os.path.splitext(os.path.basename(image_file))[0] + os.path.basename(shifted_csv_path)
        )
        
        try:
            create_updated_h5(image_file, new_h5_path, shifted_csv_path)
            print(f"Updated H5 file created at: {new_h5_path}")
        except Exception as e:
            print("Error updating H5 file:", e)

update_h5_button.on_click(on_update_h5_clicked)

h5_ui = widgets.VBox([
    widgets.HTML("<h2>Update H5 with Shifted Centers</h2>"),
    image_file_chooser_h5,
    update_h5_button,
    h5_output
])

# Combine both sections into the final UI.
ui = widgets.VBox([csv_ui, h5_ui])
display(ui)
